In [1]:
import pandas as pd
from pandas.plotting import table
import numpy as np  
import matplotlib.pyplot as plt
import openpyxl
from IPython.display import HTML
import pingouin as pg
import os

In [2]:
xl_usu = 'usu_individual_T125.xlsx'  
dfusu = pd.read_excel(xl_usu)
dfusu = dfusu.loc[:, ['CODUSU','NRO_HOGAR','COMPONENTE', 'ESTADO','V5_01_M','V5_02_M','V5_03_M']]

In [3]:
xl_hog = 'usu_hogar_T125.xlsx'  
dfhog = pd.read_excel(xl_hog)
dfhog = dfhog.loc[:, ['CODUSU','NRO_HOGAR','DECIFR','REGION','V5_01','V5_02','V5_03']]

In [4]:
# V5 = 1 si ALGUNA variable es 1, 2 solo si TODAS son 2
dfhog['V5'] = 2  # Inicializar con 2 (asumiendo que todas son 2)

# Si ALGUNA variable es 1, entonces V5 = 1
mask_alguna_1 = (dfhog['V5_01'] == 1) | (dfhog['V5_02'] == 1) | (dfhog['V5_03'] == 1)
dfhog.loc[mask_alguna_1, 'V5'] = 1

#print(dfhog['V5'].value_counts().sort_index())
dfhog.head()

,CODUSU,NRO_HOGAR,DECIFR,REGION,V5_01,V5_02,V5_03,V5
0,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2
1,TQSMNORTSHMOLTCDEGGFB00877605,1,6,42,2,2,2,2
2,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2
3,TQRMNOTTWHMMLOCDEFIAH00877819,1,12,43,2,2,2,2
4,TQRMNORQQHLMKOCDEHIBB00853810,2,1,40,2,2,2,2


In [5]:
dfusu.info()
print("--------------------------------------------------")
dfhog.info()
#dfusu.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45425 entries, 0 to 45424
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      45425 non-null  object
 1   NRO_HOGAR   45425 non-null  int64 
 2   COMPONENTE  45425 non-null  int64 
 3   ESTADO      45425 non-null  int64 
 4   V5_01_M     45425 non-null  int64 
 5   V5_02_M     45425 non-null  int64 
 6   V5_03_M     45425 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 2.4+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15984 entries, 0 to 15983
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CODUSU     15984 non-null  object
 1   NRO_HOGAR  15984 non-null  int64 
 2   DECIFR     15984 non-null  int64 
 3   REGION     15984 non-null  int64 
 4   V5_01      15984 non-null  int64 
 5   V5_02      15984 non-null  int64 
 6   V5_03 

In [6]:
#hogares: filtrar 50% más pobre (0 y 12)

print(dfhog['DECIFR'].value_counts())
dfhog_pobre = dfhog[dfhog['DECIFR'] <= 5]
print(dfhog_pobre['DECIFR'].value_counts())

#persona: filtrar por Estado 1 (ocupado), 2 (desocupado) y 3 (inactivo)
dfusu = dfusu[dfusu['ESTADO'].isin([1, 2, 3])]
print(dfusu['ESTADO'].value_counts())

print("---------------------------------------------------")
dfhog_pobre.info()
dfusu.info()

DECIFR
12    3877
1     1457
4     1350
2     1333
5     1287
3     1273
6     1221
7     1151
9     1069
8     1052
10     840
0       74
Name: count, dtype: int64
DECIFR
1    1457
4    1350
2    1333
5    1287
3    1273
0      74
Name: count, dtype: int64
ESTADO
1    20076
3    18866
2     1381
Name: count, dtype: int64
---------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 6774 entries, 0 to 15979
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CODUSU     6774 non-null   object
 1   NRO_HOGAR  6774 non-null   int64 
 2   DECIFR     6774 non-null   int64 
 3   REGION     6774 non-null   int64 
 4   V5_01      6774 non-null   int64 
 5   V5_02      6774 non-null   int64 
 6   V5_03      6774 non-null   int64 
 7   V5         6774 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 476.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 40323 entries, 0 to 45424
Data colu

In [7]:
#Juntar tablas (left hogares pobres, right individuos)
df_join_pobre = dfhog_pobre.merge(dfusu, how='left', on=['CODUSU', 'NRO_HOGAR'])
df_join_pobre.info()
df_join_pobre.head(20)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14134 entries, 0 to 14133
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14134 non-null  object
 1   NRO_HOGAR   14134 non-null  int64 
 2   DECIFR      14134 non-null  int64 
 3   REGION      14134 non-null  int64 
 4   V5_01       14134 non-null  int64 
 5   V5_02       14134 non-null  int64 
 6   V5_03       14134 non-null  int64 
 7   V5          14134 non-null  int64 
 8   COMPONENTE  14134 non-null  int64 
 9   ESTADO      14134 non-null  int64 
 10  V5_01_M     14134 non-null  int64 
 11  V5_02_M     14134 non-null  int64 
 12  V5_03_M     14134 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.4+ MB


,CODUSU,NRO_HOGAR,DECIFR,REGION,V5_01,V5_02,V5_03,V5,COMPONENTE,ESTADO,V5_01_M,V5_02_M,V5_03_M
0,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2,1,1,0,0,0
1,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2,2,3,0,0,0
2,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2,1,3,0,0,0
3,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2,2,3,0,0,0
4,TQRMNORQQHLMKOCDEHIBB00853810,2,1,40,2,2,2,2,1,3,0,0,0
5,TQRMNOTPYHMOLQCDEFMDB00877823,1,5,43,2,2,2,2,1,1,0,0,0
6,TQRMNOTPYHMOLQCDEFMDB00877823,1,5,43,2,2,2,2,2,1,0,0,0
7,TQRMNORVVHLLLPCDEFMDB00852190,1,3,43,2,2,2,2,1,3,0,0,0
8,TQRMNOQTTHKMKSCDEFMDB00857883,1,3,43,2,2,2,2,1,1,0,0,0
9,TQRMNOPXWHMOKSCDEOJAH00877825,1,3,44,2,2,2,2,1,1,0,0,0


In [8]:
# Crear UID único para el DataFrame completo
df_join_pobre['UID'] = (
    df_join_pobre['CODUSU'].astype(str) + '_' + 
    df_join_pobre['NRO_HOGAR'].astype(str) + '_' + 
    df_join_pobre['COMPONENTE'].astype(str)
)   
df_join_pobre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14134 entries, 0 to 14133
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14134 non-null  object
 1   NRO_HOGAR   14134 non-null  int64 
 2   DECIFR      14134 non-null  int64 
 3   REGION      14134 non-null  int64 
 4   V5_01       14134 non-null  int64 
 5   V5_02       14134 non-null  int64 
 6   V5_03       14134 non-null  int64 
 7   V5          14134 non-null  int64 
 8   COMPONENTE  14134 non-null  int64 
 9   ESTADO      14134 non-null  int64 
 10  V5_01_M     14134 non-null  int64 
 11  V5_02_M     14134 non-null  int64 
 12  V5_03_M     14134 non-null  int64 
 13  UID         14134 non-null  object
dtypes: int64(12), object(2)
memory usage: 1.5+ MB


In [13]:
#tomar solo columnas necesarias
df_join_pobre_nac = df_join_pobre.loc[:, ['UID', 'ESTADO', 'V5']]
df_join_pobre_nac.info()

df_join_pobre.info() #HERMES NO ESTOY PUDIENDO FILTRAR POR REGION

df_join_pobre_amba = df_join_pobre[df_join_pobre['REGION'] == '01']
df_join_pobre_amba = df_join_pobre_amba.loc[:, ['UID', 'ESTADO', 'V5']]
df_join_pobre_amba.info()

df_join_pobre_NOA = df_join_pobre[df_join_pobre['REGION'] == '40']
df_join_pobre_NOA = df_join_pobre_NOA.loc[:, ['UID', 'ESTADO', 'V5']]
df_join_pobre_NOA.info()

df_join_pobre_NEA = df_join_pobre[df_join_pobre['REGION'] == '41']
df_join_pobre_NEA = df_join_pobre_NEA.loc[:, ['UID', 'ESTADO', 'V5']]
df_join_pobre_NEA.info()

df_join_pobre_cuyo = df_join_pobre[df_join_pobre['REGION'] == '42']
df_join_pobre_cuyo = df_join_pobre_cuyo.loc[:, ['UID', 'ESTADO', 'V5']]
df_join_pobre_cuyo.info()

df_join_pobre_pamp = df_join_pobre[df_join_pobre['REGION'] == '43']
df_join_pobre_pamp = df_join_pobre_pamp.loc[:, ['UID', 'ESTADO', 'V5']]
df_join_pobre_pamp.info()

df_join_pobre_pata = df_join_pobre[df_join_pobre['REGION'] == '44']
df_join_pobre_pata = df_join_pobre_pata.loc[:, ['UID', 'ESTADO', 'V5']]
df_join_pobre_pata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14134 entries, 0 to 14133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UID     14134 non-null  object
 1   ESTADO  14134 non-null  int64 
 2   V5      14134 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 331.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14134 entries, 0 to 14133
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14134 non-null  object
 1   NRO_HOGAR   14134 non-null  int64 
 2   DECIFR      14134 non-null  int64 
 3   REGION      14134 non-null  int64 
 4   V5_01       14134 non-null  int64 
 5   V5_02       14134 non-null  int64 
 6   V5_03       14134 non-null  int64 
 7   V5          14134 non-null  int64 
 8   COMPONENTE  14134 non-null  int64 
 9   ESTADO      14134 non-null  int64 
 10  V5_01_M     14134 non-null  int64 
 11  V5_02_M     14134 non-null

In [ ]:
# --- ANÁLISIS OCUPACIÓN - SOLO CANTIDADES ---
df_ocupacion = df_join_pobre_nac.copy()

# Recodificar variables
df_ocupacion['ESTADO'] = df_ocupacion['ESTADO'].replace({1: 'Ocupado', 2: 'Desocupado', 3: 'Inactivo'})
df_ocupacion['V5'] = df_ocupacion['V5'].replace({1: 'Percibe', 2: 'No Percibe'})

# Tabla de frecuencias simple
tabla_ocupacion = pd.crosstab(
    index=df_ocupacion['V5'], 
    columns=df_ocupacion['ESTADO'],
    margins=True,
    margins_name='Total'
)

print("--- TABLA OCUPACIÓN - SOLO CANTIDADES ---")
print(tabla_ocupacion)

# Exportar
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
tabla_ocupacion.to_excel(os.path.join(output_dir, 'ocupacion_cantidades.xlsx'))
print("✅ Tabla de ocupación exportada")

--- TABLA OCUPACIÓN - SOLO CANTIDADES ---
ESTADO      Desocupado  Inactivo  Ocupado  Total
V5                                              
No Percibe         488      5905     4203  10596
Percibe            213      1960     1365   3538
Total              701      7865     5568  14134
✅ Tabla de ocupación exportada


In [45]:
# Antes de cada bloque, verifica el tamaño del DataFrame
print(f"Tamaño del DataFrame antes del análisis de actividad: {len(df_join_pobre)}")

# También verifica las frecuencias originales
print("\nFrecuencias originales de ESTADO:")
print(df_join_pobre['ESTADO'].value_counts())

print("\nFrecuencias originales de V5:")
print(df_join_pobre['V5'].value_counts())

Tamaño del DataFrame antes del análisis de actividad: 14134

Frecuencias originales de ESTADO:
ESTADO
3    7865
1    5568
2     701
Name: count, dtype: int64

Frecuencias originales de V5:
V5
2    10596
1     3538
Name: count, dtype: int64


In [48]:
# --- ANÁLISIS ACTIVIDAD - SOLO CANTIDADES ---
df_actividad = df_join_pobre.copy()

# Recodificar variables (fusionar 1 y 2 como Activo)
df_actividad['ESTADO'] = df_actividad['ESTADO'].replace({1: 'Activo', 2: 'Activo', 3: 'Inactivo'})
df_actividad['V5'] = df_actividad['V5'].replace({1: 'Percibe', 2: 'No Percibe'})

# Tabla de frecuencias simple
tabla_actividad = pd.crosstab(
    index=df_actividad['V5'], 
    columns=df_actividad['ESTADO'],
    margins=True,
    margins_name='Total'
)

print("--- TABLA ACTIVIDAD - SOLO CANTIDADES ---")
print(tabla_actividad)

# Exportar
tabla_actividad.to_excel(os.path.join(output_dir, 'actividad_cantidades.xlsx'))
print("✅ Tabla de actividad exportada")

--- TABLA ACTIVIDAD - SOLO CANTIDADES ---
ESTADO      Activo  Inactivo  Total
V5                                 
No Percibe    4691      5905  10596
Percibe       1578      1960   3538
Total         6269      7865  14134
✅ Tabla de actividad exportada
